In [ ]:
import sys
sys.path.append('../')
%load_ext autotime
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
import plotters.plot_rydberg_dynamics as rydnamics

In [ ]:
# Test run Lindblad solver
rydnamics.plot_lindblad_dynamics()

# On-Resonance Excitation to Rydberg State
We want to evaluate:
1. With CW coupling light (1064nm) to the Rydberg state, does selecting a fast probe pulse allow $\pi$ pulses to the Rydberg state with minimal loss? If so, we could potential get larger Rabi rates with less power due to the fact that we are not detuned (as long as the loss is low).
2. The on-resonance CW coupling light should make the loss very large. Therefore, we need to think about pulsing the coupling light to turn off the coupling back to the excited state and shut off the loss. We can use either a "conventional" pulse scheme or a STIRAP pulse scheme. Which one is better? Seeking less laser power, lower loss, high Rabi rate.

## Pi Pulse Times

In [ ]:
import models.rydberg_calcs as ryd
transition47 = ryd.RydbergTransition(laserWaist=25e-6, n1=6, l1=0, j1=0.5,
                 mj1=0.5, n2=7, l2=1, j2=1.5, mj2=1.5, n3=47, l3=2, j3=2.5)

transition47.print_laser_frequencies(Pp=0.01, Pc=2)

In [ ]:
coupling_powers = np.linspace(0.1, 10, 25)
probe_peak_powers = np.linspace(1e-3, 100e-3, 25)

In [ ]:
max_pi_time = transition47.get_pi_pulse_duration(probe_peak_powers[0], coupling_powers[0], resonance=False)
min_pi_time = transition47.get_pi_pulse_duration(probe_peak_powers[-1], coupling_powers[-1], resonance=False)
print("Maximum pi pulse time (ns): ", max_pi_time*1e9)
print("Minimum pi pulse time (ns): ", min_pi_time*1e9)

In [ ]:
max_pi_time = transition47.get_pi_pulse_duration(probe_peak_powers[0], coupling_powers[0], resonance=True)
min_pi_time = transition47.get_pi_pulse_duration(probe_peak_powers[-1], coupling_powers[-1], resonance=True)
print("Maximum pi pulse time (ns): ", max_pi_time*1e9)
print("Minimum pi pulse time (ns): ", min_pi_time*1e9)

In [ ]:
pi_time_opt = []
pi_time_resonant = []
for Pc in coupling_powers:
    for Pp in probe_peak_powers:
        pi_time_opt.append(transition47.get_pi_pulse_duration(Pp, Pc, resonance=False))
        pi_time_resonant.append(transition47.get_pi_pulse_duration(Pp, Pc, resonance=True))
pi_time_opt = np.reshape(pi_time_opt, (len(coupling_powers), len(probe_peak_powers)))
pi_time_resonant = np.reshape(pi_time_resonant, (len(coupling_powers), len(probe_peak_powers)))

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
s1 = ax1.imshow(pi_time_opt*1e9, aspect='auto', origin="lower", norm=LogNorm(vmin=0.1),
                extent=[np.min(probe_peak_powers), np.max(probe_peak_powers),
                        np.min(coupling_powers), np.max(coupling_powers)])
cbar = fig.colorbar(s1, ax=ax1)
cbar.set_label(r'Optimal Pi Time (ns)')
ax1.set_xlabel(r'456nm Peak Power (W)')
ax1.set_ylabel(r'1064nm Power (W)')

s2 = ax2.imshow(pi_time_resonant * 1e9, aspect='auto', origin="lower", norm=LogNorm(vmin=0.1),
                extent=[np.min(probe_peak_powers), np.max(probe_peak_powers),
                        np.min(coupling_powers), np.max(coupling_powers)])
cbar = fig.colorbar(s2, ax=ax2)
cbar.set_label(r'Resonant Pi Time (ns)')
ax2.set_xlabel(r'456nm Peak Power (W)')
ax2.set_ylabel(r'1064nm Power (W)')

plt.tight_layout()
plt.show()

Given a desired pi time, what detunings are accessible?

In [ ]:
detuning_1 = []
detuning_2 = []
for Pc in coupling_powers:
    for Pp in probe_peak_powers:
        detuning_1.append(transition47.get_pi_detuning(probe_power=Pp, couple_power=Pc, pi_time=1e-9))
        detuning_2.append(transition47.get_pi_detuning(probe_power=Pp, couple_power=Pc, pi_time=10e-9))
detuning_1 = np.reshape(detuning_1, (len(coupling_powers), len(probe_peak_powers)))
detuning_2 = np.reshape(detuning_2, (len(coupling_powers), len(probe_peak_powers)))

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
s1 = ax1.imshow(detuning_1/(2*np.pi)*1e-6, aspect='auto', origin="lower", norm=LogNorm(vmax=5e3),
                extent=[np.min(probe_peak_powers), np.max(probe_peak_powers),
                        np.min(coupling_powers), np.max(coupling_powers)])
ax1.plot(probe_peak_powers, transition47.get_balanced_laser_power(probe_power=probe_peak_powers))
cbar = fig.colorbar(s1, ax=ax1)
cbar.set_label(r'Detuning (2pi*MHz)')
ax1.set_xlabel(r'456nm Peak Power (W)')
ax1.set_ylabel(r'1064nm Power (W)')

s2 = ax2.imshow(detuning_2/(2*np.pi)*1e-6, aspect='auto', origin="lower", norm=LogNorm(vmax=5e3),
                extent=[np.min(probe_peak_powers), np.max(probe_peak_powers),
                        np.min(coupling_powers), np.max(coupling_powers)])
ax2.plot(probe_peak_powers, transition47.get_balanced_laser_power(probe_power=probe_peak_powers))
cbar = fig.colorbar(s2, ax=ax2)
cbar.set_label(r'Detuning (2pi*MHz)')
ax2.set_xlabel(r'456nm Peak Power (W)')
ax2.set_ylabel(r'1064nm Power (W)')

plt.tight_layout()
plt.show()

## Fast Probe, CW Coupling

In [ ]:
coupling_powers = np.linspace(0.001, 0.5, 20)
probe_peak_power = np.linspace(0.001, 0.01, 20)
rydnamics.plot_lindblad_fast_probe(coupling_powers=coupling_powers,
									   probe_peak_power=probe_peak_power)

## Conventional Pulse Scheme

In [ ]:
rydnamics.plot_duo_pulse(probe_duration=0, probe_delay=10e-9, probe_hold=20e-9,
        probe_peak_power=10e-3, couple_duration=0, couple_delay=15e-9,
        couple_hold=20e-9, couple_peak_power=4,
        Delta=0.0)

In [ ]:
probe_delays = np.linspace(0, 50e-9, 20)
couple_delays = np.linspace(0, 50e-9, 20)
rydnamics.plot_lindblad_duo_pulse(probe_delays=probe_delays,
                            couple_delays=couple_delays,
                            probe_peak_power=20e-3, couple_peak_power=5)

## STIRAP Pulse Scheme